In [166]:
import requests
from datetime import datetime, timedelta
from datetime import date
import pickle


In [167]:
class apartment_import_today:
    def __init__(self):
        self.headers = {"X-RapidAPI-Key": "f70d515e3fmshc7081c7447a5c31p1733e0jsnb81c8ac77ac8",
                                  "X-RapidAPI-Host": "bayut.p.rapidapi.com"}
        self.url = "https://bayut.p.rapidapi.com/properties/list"
        self.querystring = {"locationExternalIDs":"6901", #6901 refers to Downtown Dubai
                            "purpose":"for-sale",
                            "hitsPerPage":"25","page":"0",
                            "lang":"en",
                            "sort":"date-desc",
                            "categoryExternalID":"4",
                            "priceMax":"4000000",
                            "areaMin":"120",
                            "roomsMin":"2","roomsMax":"2"}
        self.result = []

        self.loop_pages_in_requests()
    
    def loop_pages_in_requests(self):
        nbPages = int(requests.get(url, headers=self.headers, params=self.querystring).json().get("nbPages"))

        for page in range(nbPages):
            self.querystring["page"] = str(page)
            response = requests.get(url, headers=self.headers, params=self.querystring).json()
            self.result.append(response)
        
        with open(str(date.today())+".txt", 'wb') as wf:
            pickle.dump(self.result, wf)



In [168]:
apartment_import_today()

In [108]:

url = "https://bayut.p.rapidapi.com/auto-complete"

querystring = {"query":"Downtown",
               "hitsPerPage":"25","page":"0",
               "lang":"en"}

headers = {
	"X-RapidAPI-Key": "f70d515e3fmshc7081c7447a5c31p1733e0jsnb81c8ac77ac8",
	"X-RapidAPI-Host": "bayut.p.rapidapi.com"
}

auto_response = requests.get(url, headers=headers, params=querystring)

In [109]:
keys = ["id","level","externalID","name"]

for hit in auto_response.json().get("hits"):
    print("\n %s - externalID: %s - levels: %s" %(hit.get("name"),hit.get("externalID"),hit.get("level")))
    for level in hit.get("hierarchy"):
        print(list( map(level.get, keys)))


 Downtown Dubai - externalID: 6901 - levels: 2
[1, 0, '5001', 'UAE']
[2, 1, '5002', 'Dubai']
[10, 2, '6901', 'Downtown Dubai']

 Al Reef Downtown - externalID: 6935 - levels: 3
[1, 0, '5001', 'UAE']
[3, 1, '6020', 'Abu Dhabi']
[481, 2, '8504', 'Al Reef']
[1593, 3, '6935', 'Al Reef Downtown']

 Downtown Views II - externalID: 9130 - levels: 4
[1, 0, '5001', 'UAE']
[2, 1, '5002', 'Dubai']
[1034, 2, '9399', "Za'abeel"]
[5320, 3, '10366', "Za'abeel 2"]
[389, 4, '9130', 'Downtown Views II']

 Ajman Downtown - externalID: 10596 - levels: 2
[1, 0, '5001', 'UAE']
[4, 1, '5385', 'Ajman']
[2206, 2, '10596', 'Ajman Downtown']

 The Central Downtown - externalID: 19598 - levels: 3
[1, 0, '5001', 'UAE']
[2, 1, '5002', 'Dubai']
[91, 2, '5785', 'Arjan']
[86505, 3, '19598', 'The Central Downtown']

 Downtown Views II Tower 1 - externalID: 17395 - levels: 5
[1, 0, '5001', 'UAE']
[2, 1, '5002', 'Dubai']
[1034, 2, '9399', "Za'abeel"]
[5320, 3, '10366', "Za'abeel 2"]
[389, 4, '9130', 'Downtown Views II']

In [79]:
for key, value in auto_response.json().items():
    print(key + ':', value)

hits: [{'id': 36, 'name': 'Dubai Marina', 'name_l1': 'دبي مارينا', 'name_l2': '迪拜码头', 'name_l3': 'Дубай Марина', 'externalID': '5003', 'slug': '/dubai/dubai-marina', 'slug_l1': '/dubai/dubai-marina', 'slug_l2': '/dubai/dubai-marina', 'slug_l3': '/dubai/dubai-marina', '_geoloc': {'lat': 25.082929715285, 'lng': 55.142790527511}, 'geography': {'lat': 25.082929715285, 'lng': 55.142790527511}, 'geo_point': [55.142790527511, 25.082929715285], 'level': 2, 'hierarchy': [{'id': 1, 'level': 0, 'externalID': '5001', 'name': 'UAE', 'name_l1': 'الإمارات', 'name_l2': '阿联酋', 'name_l3': 'ОАЭ', 'slug': '/uae', 'slug_l1': '/uae', 'slug_l2': '/uae', 'slug_l3': '/uae'}, {'id': 2, 'level': 1, 'externalID': '5002', 'name': 'Dubai', 'name_l1': 'دبي', 'name_l2': '迪拜', 'name_l3': 'Дубай', 'slug': '/dubai', 'slug_l1': '/dubai', 'slug_l2': '/dubai', 'slug_l3': '/dubai'}, {'id': 36, 'level': 2, 'externalID': '5003', 'name': 'Dubai Marina', 'name_l1': 'دبي مارينا', 'name_l2': '迪拜码头', 'name_l3': 'Дубай Марина', 'sl

In [86]:
for key, value in auto_response.json().get("hits")[3].items():
    print(key + ':', value)

id: 75477
name: Dubai Marina Moon Tower
name_l1: برج دبي مارينا مون
name_l2: 迪拜滨海月亮塔
name_l3: Дубай Марина Мун Тауэр
externalID: 15021
slug: /dubai/dubai-marina/dubai-marina-moon-tower
slug_l1: /dubai/dubai-marina/dubai-marina-moon-tower
slug_l2: /dubai/dubai-marina/dubai-marina-moon-tower
slug_l3: /dubai/dubai-marina/dubai-marina-moon-tower
_geoloc: {'lat': 25.077983870315, 'lng': 55.14121889476}
geography: {'lat': 25.077983870315, 'lng': 55.14121889476}
geo_point: [55.14121889476, 25.077983870315]
level: 3
hierarchy: [{'id': 1, 'level': 0, 'externalID': '5001', 'name': 'UAE', 'name_l1': 'الإمارات', 'name_l2': '阿联酋', 'name_l3': 'ОАЭ', 'slug': '/uae', 'slug_l1': '/uae', 'slug_l2': '/uae', 'slug_l3': '/uae'}, {'id': 2, 'level': 1, 'externalID': '5002', 'name': 'Dubai', 'name_l1': 'دبي', 'name_l2': '迪拜', 'name_l3': 'Дубай', 'slug': '/dubai', 'slug_l1': '/dubai', 'slug_l2': '/dubai', 'slug_l3': '/dubai'}, {'id': 36, 'level': 2, 'externalID': '5003', 'name': 'Dubai Marina', 'name_l1': 'دبي

In [112]:
url = "https://bayut.p.rapidapi.com/properties/list"

querystring = {"locationExternalIDs":"6901",
               "purpose":"for-sale",
               "hitsPerPage":"25","page":"0",
               "lang":"en",
               "sort":"date-desc",
               "categoryExternalID":"4",
               "priceMin":"3000000","priceMax":"4000000",
               "areaMin":"120",
               "roomsMin":"2","roomsMax":"2"}

headers = {
	"X-RapidAPI-Key": "f70d515e3fmshc7081c7447a5c31p1733e0jsnb81c8ac77ac8",
	"X-RapidAPI-Host": "bayut.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


In [139]:
for key, value in response.json().items():
    print(key + ':', value)

hits: [{'id': 5778405, 'ownerID': 1984285, 'userExternalID': '1984285', 'sourceID': 1, 'state': 'active', '_geoloc': {'lat': 25.198443, 'lng': 55.270758}, 'geography': {'lat': 25.198443, 'lng': 55.270758}, 'purpose': 'for-sale', 'price': 3400000.0, 'product': 'premium', 'productLabel': 'default', 'productScore': 3, 'rentFrequency': None, 'referenceNumber': '7ST-MT-AP-S-012', 'permitNumber': '7117731224', 'projectNumber': None, 'title': 'High Floor | SZR and Sea View | Vacant', 'title_l1': 'شقة في برج فيستا 1،برج فيستا،وسط مدينة دبي 2 غرف 3400000 درهم - 8804997', 'title_l2': '位于迪拜市中心，景观高塔，景观1号高塔 2 卧室的公寓 3400000 AED - 8804997', 'title_l3': 'Квартира в Дубай Даунтаун，Бурж Виста，Бурдж Виста 1, 2 cпальни, 3400000 AED - 8804997', 'externalID': '8804997', 'slug': 'high-floor-szr-and-sea-view-vacant-8804997', 'slug_l1': 'high-floor-szr-and-sea-view-vacant-8804997', 'slug_l2': 'high-floor-szr-and-sea-view-vacant-8804997', 'slug_l3': 'high-floor-szr-and-sea-view-vacant-8804997', 'location': [{'i

In [137]:
for key, value in response.json().get("hits")[6].items():
    print(key + ':', value)

id: 5774711
ownerID: 927631
userExternalID: 927631
sourceID: 1
state: active
_geoloc: {'lat': 25.193020803138, 'lng': 55.269647946331}
geography: {'lat': 25.193020803138, 'lng': 55.269647946331}
purpose: for-sale
price: 3990000.0
product: premium
productLabel: default
productScore: 3
rentFrequency: None
referenceNumber: PKS-2354
permitNumber: 7117457038
projectNumber: None
title: VOT | Fully Furnished | Large Layout
title_l1: شقة في بوليفارد هايتس برج 1،بوليفارد هايتس،وسط مدينة دبي 2 غرف 3990000 درهم - 8801023
title_l2: 位于迪拜市中心，BLVD塔楼，BLVD塔楼1号 2 卧室的公寓 3990000 AED - 8801023
title_l3: Квартира в Дубай Даунтаун，Бульвар Хейтс，BLVD Хайтс Тауэр 1, 2 cпальни, 3990000 AED - 8801023
externalID: 8801023
slug: vot-fully-furnished-large-layout-8801023
slug_l1: vot-fully-furnished-large-layout-8801023
slug_l2: vot-fully-furnished-large-layout-8801023
slug_l3: vot-fully-furnished-large-layout-8801023
location: [{'id': 1, 'level': 0, 'externalID': '5001', 'name': 'UAE', 'name_l1': 'الإمارات', 'name_l

In [131]:
keys = ["level","externalID","name"]
for hit in response.json().get("hits"):
    print("\n")
    for x in range(2,len(hit.get("location"))):
        item = hit.get("location")[x]
        print(list( map(item.get, keys)))



[2, '6901', 'Downtown Dubai']
[3, '8219', 'Burj Vista']
[4, '10133', 'Burj Vista 1']


[2, '6901', 'Downtown Dubai']
[3, '8219', 'Burj Vista']
[4, '10133', 'Burj Vista 1']


[2, '6901', 'Downtown Dubai']
[3, '8484', 'BLVD Heights']
[4, '9881', 'BLVD Heights Tower 1']


[2, '6901', 'Downtown Dubai']
[3, '5131', 'South Ridge']
[4, '8795', 'South Ridge 1']


[2, '6901', 'Downtown Dubai']
[3, '5099', 'The Lofts']
[4, '8741', 'The Lofts West']


[2, '6901', 'Downtown Dubai']
[3, '5131', 'South Ridge']
[4, '8800', 'South Ridge 6']


[2, '6901', 'Downtown Dubai']
[3, '8484', 'BLVD Heights']
[4, '9881', 'BLVD Heights Tower 1']


[2, '6901', 'Downtown Dubai']
[3, '11865', 'Mon Reve']


[2, '6901', 'Downtown Dubai']
[3, '8219', 'Burj Vista']
[4, '8227', 'Burj Vista 2']


[2, '6901', 'Downtown Dubai']
[3, '10342', 'Opera Grand']


[2, '6901', 'Downtown Dubai']
[3, '5131', 'South Ridge']
[4, '8799', 'South Ridge 5']


[2, '6901', 'Downtown Dubai']
[3, '5587', 'Old Town']
[4, '6714', 'Yansoon']
[

In [73]:
x = response.json().get("hits")[0].get("updatedAt")
print(datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

2024-03-25 12:58:39


In [75]:
x = response.json().get("hits")[24].get("updatedAt")
print(datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

2024-03-23 00:12:26


In [76]:
x = response2.json().get("hits")[0].get("updatedAt")
print(datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))

2024-03-23 00:08:23


In [77]:
x = response2.json().get("hits")[24].get("createdAt")
print(datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))


2024-03-23 00:00:32


In [138]:
for hit in response.json().get("hits"):
    createdAt = str(datetime.fromtimestamp(hit.get("createdAt")).strftime('%Y-%m-%d %H:%M:%S'))
    updatedAt = str(datetime.fromtimestamp(hit.get("updatedAt")).strftime('%Y-%m-%d %H:%M:%S'))
    reactivatedAt = str(datetime.fromtimestamp(hit.get("reactivatedAt")).strftime('%Y-%m-%d %H:%M:%S'))

    print("created: %s - updated: %s - reactivated: %s " %(createdAt, updatedAt, reactivatedAt))

created: 2024-03-27 18:29:39 - updated: 2024-03-28 00:35:23 - reactivated: 2024-03-27 18:29:39 
created: 2024-03-27 14:43:17 - updated: 2024-03-28 00:20:23 - reactivated: 2024-03-27 14:43:17 
created: 2024-03-27 13:31:15 - updated: 2024-03-28 00:08:19 - reactivated: 2024-03-27 13:31:15 
created: 2024-03-27 13:02:15 - updated: 2024-03-28 00:35:22 - reactivated: 2024-03-27 13:02:15 
created: 2024-03-27 12:46:24 - updated: 2024-03-28 00:35:23 - reactivated: 2024-03-27 12:46:24 
created: 2024-03-27 10:48:49 - updated: 2024-03-28 00:08:19 - reactivated: 2024-03-27 10:48:49 
created: 2024-03-27 10:16:45 - updated: 2024-03-28 00:08:19 - reactivated: 2024-03-27 10:16:45 
created: 2024-03-26 11:52:43 - updated: 2024-03-27 00:12:23 - reactivated: 2024-03-26 11:52:43 
created: 2024-03-25 14:37:35 - updated: 2024-03-28 08:48:37 - reactivated: 2024-03-25 14:37:35 
created: 2024-03-25 14:07:31 - updated: 2024-03-26 00:16:21 - reactivated: 2024-03-25 14:07:31 
created: 2024-03-25 13:41:25 - updated: 

In [159]:
response.json()

{'hits': [{'id': 5778405,
   'ownerID': 1984285,
   'userExternalID': '1984285',
   'sourceID': 1,
   'state': 'active',
   '_geoloc': {'lat': 25.198443, 'lng': 55.270758},
   'geography': {'lat': 25.198443, 'lng': 55.270758},
   'purpose': 'for-sale',
   'price': 3400000.0,
   'product': 'premium',
   'productLabel': 'default',
   'productScore': 3,
   'rentFrequency': None,
   'referenceNumber': '7ST-MT-AP-S-012',
   'permitNumber': '7117731224',
   'projectNumber': None,
   'title': 'High Floor | SZR and Sea View | Vacant',
   'title_l1': 'شقة في برج فيستا 1،برج فيستا،وسط مدينة دبي 2 غرف 3400000 درهم - 8804997',
   'title_l2': '位于迪拜市中心，景观高塔，景观1号高塔 2 卧室的公寓 3400000 AED - 8804997',
   'title_l3': 'Квартира в Дубай Даунтаун，Бурж Виста，Бурдж Виста 1, 2 cпальни, 3400000 AED - 8804997',
   'externalID': '8804997',
   'slug': 'high-floor-szr-and-sea-view-vacant-8804997',
   'slug_l1': 'high-floor-szr-and-sea-view-vacant-8804997',
   'slug_l2': 'high-floor-szr-and-sea-view-vacant-8804997',
 